## Resource-Constrained Project Scheduling with Blocking Times No-Preemption

### Problem Definiton

Tasks have flexible execution windows (preemption allowed based on availability) but strict finish-to-start precedence constraints. Each task requires specific quantities of resources from disjoint candidate sets. Unlike standard duration-based scheduling, tasks here are defined by size, allowing them to split across resource breaks (e.g., weekends) automatically. Formally, the problem is defined as follows: Let $\mathcal{N}=\{1,\ldots,N\}$ be a set of tasks. Let $S\in\mathbb{N}^{|\mathcal{N}|}$ be a vector of task sizes (total effort required). Let $\mathcal{E}$ be a set of precedences. Let $\mathcal{R}=\{1,\ldots,M\}$ be a set of resources. Let $\bar{u}_r(t): \mathbb{N} \to \{0,1\}$ be the availability function (intensity) of resource $r$. For each task $i$, let $\mathcal{K}_i$ be resource requirement indices, where each requirement has a candidate set $\mathcal{C}_{i,k}$ and a quantity $Q_{i,k}$.

### CP Optimizer Formulation

$$
\begin{aligned}
\min \quad
& \max_{i \in \mathcal{N}} (\operatorname{end}(T_i))
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \operatorname{endBeforeStart}(T_i, T_j),
\qquad & \forall (i, j) \in \mathcal{E}
\quad & \text{(2)} \\[2mm]
& \sum_{r \in \mathcal{C}_{i,k}} \operatorname{presenceOf}(A_{i,r}) = Q_{i,k},
\qquad & \forall i \in \mathcal{N}, \forall k \in \mathcal{K}_i
\quad & \text{(3)} \\[2mm]
& \operatorname{startAtStart}(T_i, A_{i,r}) \land \operatorname{endAtEnd}(T_i, A_{i,r}),
\qquad & \forall i \in \mathcal{N}, \forall k \in \mathcal{K}_i, \forall r \in \mathcal{C}_{i,k}
\quad & \text{(4)} \\[2mm]
& \operatorname{noOverlap}(\{A_{i,r} \mid i \in \mathcal{N} \text{ s.t. } r \in \bigcup \mathcal{C}\}),
\qquad & \forall r \in \mathcal{R}
\quad & \text{(5)} \\[2mm]
& \operatorname{sizeOf}(A_{i,r}) = \int_{\operatorname{start}(A_{i,r})}^{\operatorname{end}(A_{i,r})} \mathcal{F}_r(t) \, dt,
\qquad & \forall i \in \mathcal{N}, \forall k \in \mathcal{K}_i, \forall r \in \mathcal{C}_{i,k}
\quad & \text{(6)} \\[2mm]
& \mathcal{F}_r = \operatorname{CpoStepFunction}(\text{steps}=\{(t_0, v_0), \dots, (t_m, v_m)\}),
\qquad & \forall r \in \mathcal{R}
\quad & \text{(7)} \\[2mm]
& T_i: \text{mandatory interval var, size } S_i,
\qquad & \forall i \in \mathcal{N}
\quad & \text{(8a)} \\[1mm]
& A_{i,r}: \text{optional interval var, size } S_i, \text{ intensity } \mathcal{F}_r,
\qquad & \forall i \in \mathcal{N}, \forall k \in \mathcal{K}_i, \forall r \in \mathcal{C}_{i,k}
\quad & \text{(8b)}
\end{aligned}
$$

Objective
* **(1)** Minimize Makespan: The goal is to minimize the end time of the latest completing task in the project.

Modeling Constraints
* **(2)** Precedence Relations: Enforces strictly that task $j$ cannot start until task $i$ has finished.
* **(3)** Multi-Resource Quantity Assignment: Ensures that for every requirement $k$ of task $i$, exactly $Q_{i,k}$ resources are selected from the candidate set $\mathcal{C}_{i,k}$.
* **(4)** Temporal Synchronization: Enforces that if a resource $r$ is selected (assigned) to a task, it must execute exactly parallel to the master task interval $T_i$ (same Start and End).
* **(5)** Resource Capacity: The `noOverlap` constraint ensures that a specific resource $r$ processes at most one task at a time.
* **(6)** Intensity Integration: Mathematically links the Size of the assignment to the integral of the efficiency function $\mathcal{F}_r$. This implies that if $\mathcal{F}_r(t)=0$, the interval must elongate to accumulate the required Size $S_i$.

Variable and Function Definitions
* **(7)** $\mathcal{F}_r$: A `CpoStepFunction` object representing the efficiency of resource $r$. It is defined by a list of steps `(x, val)` indicating the value of the function starting at time `x`.
* **(8a)** $T_i$: A mandatory interval variable representing the execution of task $i$. It is defined by Size $S_i$, not fixed length.
* **(8b)** $A_{i,r}$: An optional interval variable representing the assignment of task $i$ to resource $r$. It is defined by Size $S_i$ and is explicitly linked to the intensity function $\mathcal{F}_r$.

Parameters (Input Data)
* $\mathcal{N}$: The set of tasks.
* $\mathcal{E}$: The set of precedence relations.
* $\mathcal{R}$: The set of discrete resources.
* $S_i$: The fixed Size (effort in man-hours/machine-hours) of task $i$.
* $steps$: The list of tuples $(t, v)$ defining the availability calendar for each resource.
* $\mathcal{K}_i$: The set of requirement indices for task $i$ (e.g., $\{1, 2\}$ if a task needs both a type 1 and a type 2 resource).
* $\mathcal{C}_{i,k}$: The disjoint set of candidate resources for requirement $k$ of task $i$.
* $Q_{i,k}$: The quantity of resources required from set $\mathcal{C}_{i,k}$.

### DOCPLEX Implementation

#### Imports

In [1]:
from docplex.cp.model import *
import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt

#### Parsing the data file

In [2]:
def next_line(f):
    """Read next non-empty, non-comment line"""
    line = None
    while not line:
        raw = f.readline()
        if not raw:
            return None
        line = raw.strip()
        if line.startswith('#'):
            line = None
    return [int(v) for v in line.split()]

In [3]:
filename = "../data/rcpspblocking/01.data"

with open(filename, "r") as f:
    N, M = next_line(f)
    TASKS = []
    for _ in range(N):
        task_data = next_line(f)
        task_id = task_data[0]
        size = task_data[1]
        num_reqs = task_data[2]
        requirements = []
        idx = 3
        for _ in range(num_reqs):
            num_candidates = task_data[idx]
            quantity = task_data[idx + 1]
            candidates = task_data[idx + 2:idx + 2 + num_candidates]
            requirements.append((num_candidates, quantity, candidates))
            idx += 2 + num_candidates
        TASKS.append((task_id, size, num_reqs, requirements))
    RESOURCES = []
    for _ in range(M):
        res_data = next_line(f)
        res_id = res_data[0]
        num_steps = res_data[1]
        steps = [(res_data[2 + 2*i], res_data[2 + 2*i + 1]) for i in range(num_steps)]
        RESOURCES.append((res_id, steps))
    prec_data = next_line(f)
    num_prec = prec_data[0]
    PRECEDENCES = [(prec_data[1 + 2*i], prec_data[1 + 2*i + 1]) for i in range(num_prec)]

#### Prepare data structures

In [ ]:
S = {t[0]: t[1] for t in TASKS} # Extract task sizes
# Extract requirements: K_i = list of requirement indices
# C_{i,k} = candidate set for requirement k of task i
# Q_{i,k} = quantity needed from candidate set
K = {t[0]: list(range(t[2])) for t in TASKS}
C = {(t[0], k): req[2] for t in TASKS for k, req in enumerate(t[3])}
Q = {(t[0], k): req[1] for t in TASKS for k, req in enumerate(t[3])}
E = PRECEDENCES # Precedences
F = {} # Create step functions for resource availability
for res_id, steps in RESOURCES:
    F[res_id] = CpoStepFunction(steps=steps)

Loaded instance with 8 tasks, 4 resources, 12 precedences


In [7]:
print(f"Loaded instance with {N} tasks, {M} resources, {len(E)} precedences")

Loaded instance with 8 tasks, 4 resources, 12 precedences


#### Create model and variables

#### Add constraints and define objective

#### Solve the model

#### Visualisation

## Aditional Resources

#### [ForbidExtent()](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.forbid_extent) vs [Stepwise Functions](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.function.py.html#docplex.cp.function.CpoStepFunction)

A. Stepwise Function (`CpoStepFunction`)
* **Type:** Data Structure (Passive).
* **Definition:** A function $f(t)$ that defines values over time.
* **Role:** It acts as a **Map**. It describes the environment (e.g., "The machine is broken from 10:00 to 12:00").
* **Effect:** On its own, it does **nothing** to the schedule. It just holds information.

B. Forbid Extent (`forbid_extent`)
* **Type:** Constraint (Active).
* **Definition:** A rule applied to the model.
* **Role:** It acts as the **Enforcer**. It looks at a Stepwise Function and applies a restriction.
* **Effect:** It forces the solver to ensure a task **never overlaps** with any time segment where the Stepwise Function has a value of `0`.

In [5]:
from docplex.cp.model import CpoModel, CpoStepFunction, interval_var, forbid_extent

mdl = CpoModel()

# --- PART 1: The Map (Stepwise Function) ---
# We define that a resource is unavailable (0) between time 10 and 20.
# This is just data. The solver doesn't care yet.
availability = CpoStepFunction()
availability.set_value(0, 100)   # Available
availability.set_value(10, 0)    # UNAVAILABLE (The "Hole")
availability.set_value(20, 100)  # Available again

# --- PART 2: The Actor (Interval Variable) ---
# A task that takes 5 units of time.
task = interval_var(length=5, name="MyTask")

# --- PART 3: The Rule (Forbid Extent) ---
# This effectively says: "Make sure 'task' does not fall into the 'Hole'."
# Without this line, the solver could schedule the task at t=12.
mdl.add(forbid_extent(task, availability))

mdl.solve()

TypeError: CpoFunction.set_value() missing 1 required positional argument: 'v'

[ICAPS 2017: Video Tutorial – Philippe Laborie: Introduction to CP Optimizer for Scheduling](https://www.youtube.com/watch?v=-VY7QTnSAio), [Slides from the video](https://icaps17.icaps-conference.org/tutorials/T3-Introduction-to-CP-Optimizer-for-Scheduling.pdf)

slide 42: interval variable size, length adn intensity funciton

what for? 
- modeling cases where the intensity of work is not the same duting the whole interval and the interval requires some quantity of work to be done before completion

- activities that are suspended during some time periods (e.g. week-end, vacation)

- https://www.ibm.com/docs/en/icos/22.1.2?topic=scheduling-interval-variables

How to model that?

- Stretch tasks across closed periods (simple “calendar” via intensity)
- When to use: Tasks are not preemptable and simply “pause” during closed time (e.g., a 6-hour job that starts at 10:00 spans over a 12:00–13:00 lunch and finishes at 17:00).

Make each task’s processing requirement the size.

Give tasks an intensity step function that is 100 when working and 0 when blocked.
CP Optimizer will automatically lengthen the interval to cover the size only during periods with intensity > 0.

In [ ]:
mdl = CpoModel()

# Example data
jobs = ["A", "B"]
durations = {"A": 6, "B": 4}  # required work time in "work units" (e.g. hours)

# Define a step function for working hours
# open 8:00–12:00, 13:00–17:00 (minutes since midnight)
work_cal = CpoStepFunction()
work_cal.set_value(0, 8*60, 0)      # before 8:00 off
work_cal.set_value(8*60, 12*60, 100)
work_cal.set_value(12*60, 13*60, 0)
work_cal.set_value(13*60, 17*60, 100)
work_cal.set_value(17*60, 24*60, 0)

# Create interval variables with that intensity
tasks = {j: mdl.interval_var(size=durations[j], intensity=work_cal, name=j)
         for j in jobs}

# Single machine
mdl.add(mdl.no_overlap(list(tasks.values())))

# Example objective (minimize makespan)
mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))


Forbid any overlap with closed periods (forbidExtent)

When to use: You want to prohibit a task from covering any closed time at all (it must sit entirely inside open windows).

 - Build a “closed” step function that is 1 during blocked time and 0 otherwise.

- Use forbidExtent so [start, end) of each interval never intersects a blocked segment.

In [ ]:
mdl = CpoModel()

jobs = ["A", "B"]
durations = {"A": 6, "B": 4}

# Define a "closed" step function: 1 during blocked periods, 0 otherwise
closed = CpoStepFunction()
closed.set_value(0, 8*60, 1)
closed.set_value(8*60, 12*60, 0)
closed.set_value(12*60, 13*60, 1)
closed.set_value(13*60, 17*60, 0)
closed.set_value(17*60, 24*60, 1)

tasks = {j: mdl.interval_var(size=durations[j], name=j) for j in jobs}

# Forbid tasks from overlapping blocked times
for j in jobs:
    mdl.add(mdl.forbid_extent(tasks[j], closed))

mdl.add(mdl.no_overlap(list(tasks.values())))
mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))


Insert downtime as fixed intervals and share the resource (noOverlap with extra intervals)

When to use: You want downtime to consume the same resource just like a job (e.g., planned maintenance blocks the machine).

- Create fixed, present interval variables for each blocking window.

- Put both jobs and downtime intervals into the same noOverlap.

In [ ]:
mdl = CpoModel()

jobs = ["A", "B"]
durations = {"A": 6, "B": 4}

# Known downtime intervals (start, end)
downtimes = [(12*60, 13*60)]  # 12:00–13:00

tasks = {j: mdl.interval_var(size=durations[j], name=j) for j in jobs}

# Create fixed blocking intervals
blocks = []
for i, (s, e) in enumerate(downtimes):
    iv = mdl.interval_var(start=s, end=e, name=f"block_{i}")
    mdl.add(mdl.presence_of(iv) == 1)
    blocks.append(iv)

# Combine jobs + blocks in same machine constraint
mdl.add(mdl.no_overlap(list(tasks.values()) + blocks))

mdl.add(mdl.minimize(mdl.max([mdl.end_of(t) for t in tasks.values()])))

forbidStart(), forbidEnd(), forbidExtent()